In [ ]:
conda install -c pytorch torchtext

In [1]:
import spacy

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import random
import math
import os

import torch.nn.functional as F

In [2]:
conda install -c conda-forge spacy
#python -m spacy download de_core_news_sm

SyntaxError: invalid syntax (<ipython-input-2-028caf04b738>, line 1)

In [15]:
python -m spacy download de_core_news_sm #this needs to be installed via anaconda prompt (admin model)

SyntaxError: invalid syntax (<ipython-input-15-abd722520997>, line 1)

In [2]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')


In [16]:
conda install -c conda-forge spacy

^C

Note: you may need to restart the kernel to use updated packages.


In [ ]:
python -m spacy download en

In [ ]:
$python -m spacy download de_core_news_sm

In [3]:
# this is tokenize sentenses for english language and german language
def process_en(sent):
    return [tok.text for tok in spacy_en.tokenizer(sent)]

def process_de(sent):
    return [tok.text for tok in spacy_de.tokenizer(sent)]

In [4]:
# this is a tool to convert all sentences to tokenized one with "sos" token at the begining and "eos" at the end
# and all in lower case

Source = Field(tokenize=process_de,init_token='<sos>',eos_token='<eos>',lower=True)
Target = Field(tokenize=process_en,init_token='<sos>',eos_token='<eos>',lower=True)

In [5]:
# So we use multi30k dataset and apply fields for the tokenization
train_data,valid_data,test_data = Multi30k.splits(exts=('.de','.en'),fields=(Source,Target))

In [6]:
for x,y in train_data:
    print(x)
    print(y)
    break

TypeError: cannot unpack non-iterable Example object

In [6]:
#skip
len(train_data)

29000

In [16]:
len(train_data[1].src)
len(train_data[664].src)

11

In [8]:
#skip
for x in train_data:
    print(x.src)
    print(x.trg)
    
    break

['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


In [9]:
Source.build_vocab(train_data,min_freq=2)
Target.build_vocab(train_data,min_freq=2)

In [18]:
batch_size=128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
train_iterator,valid_iterator,test_iterator = BucketIterator.splits(
    (train_data,valid_data,test_data),batch_size=batch_size,device=device)

In [24]:
# for x in train_iterator:
#     print(len(x.src))
for x,y in enumerate(train_iterator):
    source = y.src
    target = y.trg 

AttributeError: 'Field' object has no attribute 'vocab'

In [26]:
class Encoder(nn.Module):
    def __init__(self,input_dim,embedding_dim,enc_hidden_dim,dec_hidden_dim,dropout):
        super().__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.enc_hidden_dim = enc_hidden_dim #!!!!!!!!!!!!!!!!
        self.dec_hidden_dim = dec_hidden_dim #!!!!!!!!!!!!!!!!
        self.embedding = nn.Embedding(input_dim,embedding_dim)
        self.rnn = nn.GRU(embedding_dim,enc_hidden_dim,bidirectional=True) #!!!!!!!!!!!!!!!!
        self.fc = nn.Linear(enc_hidden_dim * 2, dec_hidden_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self,source_data):
        embedded = self.dropout(self.embedding(source_data)) #shape of source_data: [sent len, batch size]
        outputs,hidden = self.rnn(embedded) #shape of embedded: [sent len, batch size, emb dim]
        # please note here is we dont enter any hidden state as input/inital hidden state, it will take zero as by default
        #shape of outputs: [sent len, batch size, hid dim* n directions]
        #shape of hidden: [n layers*n directions, batch size, hid dim]
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1))) #!!!!!!!!!!!!!!!!
        return outputs,hidden #!!!!!!!!!!!!!!!!
        ##since hidden now is the concat of its components and it gets fed into linear layer,
        ##the shape is now changed to [batch_size,dec hid dim]
        ### It is important that we now convert our encoder hidden state tensor size to decoder hidden state tensor size
        ### using a linear layer

In [27]:
### whole point here is we will combine previous hidden state tensor (in decoder) with encoder output tensors and
### pass it through a linear layer then reshape and apply softmax to output the probabilities
### attention length is equal to the length of source sentence
### by doing that we just pretty much concat each decoder previous hidden state with all encoder output states 
### since encoder output states is its hidden state at each timestep

class Attention(nn.Module):
    def __init__(self,enc_hidden_dim,dec_hidden_dim):
        super().__init__()
        self.enc_hidden_dim = enc_hidden_dim
        self.dec_hidden_dim = dec_hidden_dim
        self.attn = nn.Linear((enc_hidden_dim*2) + dec_hidden_dim, dec_hidden_dim)
        ## the dec_hidden_dim is for the hidden state from encoder since it passed through linear layer (enc_hidden_dim * 2, dec_hidden_dim)
        ## and enc_hidden_dim*2 is for encoder outputs with shape [sent len, batch size, hid dim* n directions] (see above)

        self.vec = nn.Parameter(torch.rand(dec_hidden_dim))
        
    def forward(self,hidden,encoder_outputs): 
        # plz note the hidden above refers to hidden state in decoder and encoder_ouputs is for output matrix from encoder
        # so we try to create a small nerual network between hidden state of decoder and encoder outputs
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1,src_len,1)
        # previous hidden from decoder is same shape as hidden state returned from Encoders: [batch_size, dec hid dim] after the linear layer
        # so we make it repeat for the sentence length and add as second dimension
        
        encoder_outputs = encoder_outputs.permute(1,0,2)
        association = torch.tanh(self.attn(torch.cat((hidden,encoder_outputs),dim=2)))
        ## so what we do here is repeat hidden with number equal to length of sentence
        ## then we concatenate this reshaped hidden with outputs from encoder and this is why we have (enc_hidden_dim*2) + dec_hidden_dim,
        ## then we pass this combined tensor into attn (linear layer) to get the associate tensor
        
        association = association.permute(0,2,1)
        vec = self.vec.repeat(batch_size,1).unsqueeze(1)
        attention = torch.bmm(vec,association).squeeze(1)
        return F.softmax(attention,dim=1)

In [28]:
# please note this Decoder is a bit different from conventional Seq2Seq model, where each timestep in decoder takes hidden state
# from previous step and current input. This new mothod takes in input combine (concate) with context state 
#(hidden state from encoder) at each timestep and also takes in hidden state from previous step
# also you use embedded input + current hidden state + context victor to feed into a linear layer for prediction
class Decoder(nn.Module):
    def __init__(self,output_dim,embedding_dim,enc_hid_dim,dec_hid_dim, dropout,attention): 
        super().__init__()
        self.embedding_dim = embedding_dim
        self.output_dim = output_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim,embedding_dim)
        self.rnn = nn.GRU((enc_hid_dim*2)+embedding_dim, dec_hid_dim) #since the bidirection hidden state from recoder has been 
                                                        #passed through linear layer to make it the size of decoder hidden state
        #please note that enc_hid_dim*2 above is for the weighted matrix: attention x encoder hidden state
        #and the GRU input will be concat(embedded input + weighted) as well as previous hidden state
        
        self.out = nn.Linear((enc_hid_dim*2) + dec_hid_dim + embedding_dim,output_dim)
        #please note we pass embedded input, weighted and current hidden state into linear layer for prediction
        
        self.dropout = nn.Dropout(dropout)
    def forward(self,data,hidden,encoder_outputs): #hidden equals to context at inital timestep, then hidden will be udpated
        #input shape: [batch size]
        #hidden: [n layers*n directions, batch size, hid dim]
        
        data = data.unsqueeze(0)
        #input: [1, batch size] !!!this because the data hasnt been passed to embedding layer therefore no featrues dimensions
        # so we can see from the Seq2Seq secton below that the data will pass to the decoder one word by one word
        embedded = self.dropout(self.embedding(data))
        #embedded: [1, batch size, emb dim]
        #####!!!!!!!!!!!!!!! please note this will be our decoder input dimension for capstone project as our network does not
        #####!!!!!!!!!!!!!!  have embedding layer
        
        a = self.attention(hidden,encoder_outputs)
        # a shape: [batch size, source length]
        # So basically every step we kind of have match b/w current hidden and encoder ouputs
        a = a.unsqueeze(1)
        # a shape: [batch size, 1, source length]
        
        encoder_outputs = encoder_outputs.permute(1,0,2)
        #shape is now [batch size, src ent len, enc hid dim*2] since it is bidirectional GRU
        
        weighted = torch.bmm(a, encoder_outputs)
        #weighted shape [batch_size,1,enc hid dim *2]
        weighted = weighted.permute(1,0,2)
        #new shape is now [1,batch_size,enc hid dim*2]
        
        rnn_input = torch.cat((embedded,weighted),dim=2)
        
        
        
        output, hidden = self.rnn(rnn_input,hidden.unsqueeze(0))
        # please note that we use context vector as inital hidden state
        # after that hidden will just be hidden state from the above rnn
        # output shape: [sent len, batch size, dec hid dim * n direction] therefore here is [1, batch size, dec hid dim]
        # hidden shape: [n layers * n direction, batch size, dec hid dim] therefore here is [1, batch size, dec hid dim]
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        #please note above three lines is to remove unneeded dimensions
        
        final_output = self.out(torch.cat((output,weighted,embedded),dim=1))
        #size: [batch size, output dim]
        
        
        return final_output, hidden.squeeze(0)
        
        

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self,source,target,teacher_forcing_ratio=0.5):
        #source:[sent len, batch size]
        #target:[sent len, batch size]
        
        batch_size = target.shape[1]
        max_len = target.shape[0]
        target_voc_size = self.decoder.output_dim
        outputs = torch.zeros(max_len,batch_size,target_voc_size).to(self.device)
        encoder_outputs,hidden = self.encoder(source)

        input = target[0,:] # this is equalivalent to target[0] which is the first word of in a batch
        for t in range(1,max_len):
            output,hidden = self.decoder(input,hidden,encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio # will return ture if teacher ratio is greater than random #
            top1 = output.max(1)[1]
            input = (target[t] if teacher_force else top1)
            # so this code will decide whether you use the next target word or our prediction as next input
        
        return outputs
        
        

In [30]:
Input_dim = len(Source.vocab)
Output_dim = len(Target.vocab)
Embedding_dim=300
enc_Hidden_dim=512
dec_Hidden_dim=512 # for my example, i just will the same hidden dim for encoder and decoder
Mydropout=0.5

attn = Attention(enc_Hidden_dim,dec_Hidden_dim)
encoder = Encoder(Input_dim,Embedding_dim,enc_Hidden_dim,dec_Hidden_dim,Mydropout)
decoder = Decoder(Output_dim,Embedding_dim,enc_Hidden_dim,dec_Hidden_dim,Mydropout,attn)

model = Seq2Seq(encoder,decoder,device).to(device)

AttributeError: 'Field' object has no attribute 'vocab'

In [27]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7854, 300)
    (rnn): GRU(300, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
    )
    (embedding): Embedding(5893, 300)
    (rnn): GRU(1324, 512)
    (out): Linear(in_features=1836, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [28]:
opt2 = torch.optim.Adam(combine_model.parameters(),lr=0.01)   

In [29]:
for a in range(5):
    model.train()
    epoch_loss=0
    for x,y in enumerate(train_iterator):
        source = y.src
        target = y.trg #[sent len, batch size]
        opt.zero_grad()
        output = model(source,target)  #output:[sent len, batch_size,output_dim]
        loss = criterion(output[1:].view(-1,output.shape[2]),target[1:].view(-1)) #this is to flatten both output and target
        # new target is shaped as [(sent len -1) * batch size]
        # new output is shaped as [(sent len -1) * batch size, output dim]
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
        opt.step()
        opti_scheduler.step()
        epoch_loss += loss.item()
        batch_PPL = math.exp(loss.item())
        
        if (x+1)%2==0:
            print('batch #{} loss: {:.5f} and PPL: {:.5f} with lr rate: {}'.format(x+1,loss,batch_PPL,opti_scheduler.get_last_lr()))
    
    print('epoch_loss for epoch # {} is {:.5f} and PPL: {:.5f}'.format(a+1,epoch_loss/len(train_iterator),math.exp(epoch_loss/len(train_iterator))))
    

batch #2 loss: 6.92910 and PPL: 1021.56932 with lr rate: [0.005]
batch #4 loss: 6.77527 and PPL: 875.91582 with lr rate: [0.005]
batch #6 loss: 8.07998 and PPL: 3229.18372 with lr rate: [0.005]
batch #8 loss: 6.65731 and PPL: 778.45668 with lr rate: [0.005]
batch #10 loss: 6.57025 and PPL: 713.54483 with lr rate: [0.005]
batch #12 loss: 6.45939 and PPL: 638.67145 with lr rate: [0.005]
batch #14 loss: 6.14111 and PPL: 464.56959 with lr rate: [0.005]
batch #16 loss: 5.63077 and PPL: 278.87736 with lr rate: [0.005]
batch #18 loss: 5.67563 and PPL: 291.67331 with lr rate: [0.005]
batch #20 loss: 5.35086 and PPL: 210.78942 with lr rate: [0.005]


KeyboardInterrupt: 

In [21]:
#evaluation

model.eval()
epoch_loss=0
with torch.no_grad():
    for x,y in enumerate(valid_iterator):
        source = y.src
        target = y.trg
        output = model(source,target)
        loss = criterion(output[1:].view(-1,output.shape[2]),target[1:].view(-1))
        epoch_loss += loss.item()
    
    print('Validation loss: {:.5f} and PPL loss: {:.8f} and torch PPL: {:.8f}'.format(epoch_loss / len(valid_iterator),math.exp(epoch_loss / len(valid_iterator)),
                                                                                     torch.exp(torch.tensor(epoch_loss / len(valid_iterator)))))
    

Validation loss: 3.48675 and PPL loss: 32.67957922 and torch PPL: 32.67957687


In [36]:
#test
model.eval()
epoch_loss=0
with torch.no_grad():
    for x,y in enumerate(test_iterator):
        source = y.src
        target = y.trg
        output = model(source,target)
        loss = criterion(output[1:].view(-1,output.shape[2]),target[1:].view(-1))
        epoch_loss += loss.item()
    
    print('Test loss: {:.5f} and PPL loss: {:.8f} and torch PPL: {:.8f}'.format(epoch_loss / len(test_iterator),math.exp(epoch_loss / len(test_iterator)),
                                                                                     torch.exp(torch.tensor(epoch_loss / len(test_iterator)))))

Test loss: 3.43148 and PPL loss: 30.92242717 and torch PPL: 30.92243004


In [28]:
import numpy as np
a = torch.tensor([[1,2,3],[4,5,6]])
a.shape

torch.Size([2, 3])

In [30]:
a[0]

tensor([1, 2, 3])

In [5]:
a[0,:]

tensor([1, 2, 3])

In [6]:
a[1]

tensor([4, 5, 6])

In [7]:
a.view(-1,2).shape

torch.Size([3, 2])

In [9]:
a.repeat(1,2,1)

tensor([[[1, 2, 3],
         [4, 5, 6],
         [1, 2, 3],
         [4, 5, 6]]])

In [22]:
a.repeat(2,1,1)

tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]]])

In [25]:
aa = torch.tensor([1,2,3])

In [27]:
print(F.softmax(aa),dim=0)

C:\Users\Bob\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


RuntimeError: "softmax_lastdim_kernel_impl" not implemented for 'Long'

In [44]:
b = torch.tensor([1.2,2.3,3.2])

In [47]:
print(F.softmax(b,dim=0))

tensor([0.0878, 0.2637, 0.6485])
